In [ ]:
print(outputs_x_test_tensored)
print('!'*100)
print(y_prob)
# y_prob instead of outputs_x_test_tensored

In [ ]:
            x_train_tensored, y_train_tensored, x_test_tensored, y_test_tensored = PYTORCHUTILS.get_tensored_data(x_train, y_train, x_test, y_test)
            trainset = torch_data_utils.TensorDataset(x_train_tensored, y_train_tensored)
            testset = torch_data_utils.TensorDataset(x_test_tensored, y_test_tensored)

            nnptc_params = algoSetting.get_nnptc_params_dict()[0]
            layers_for_network = PYTORCHUTILS.get_layers_for_network_module(nnptc_params, task_type = "CLASSIFICATION", first_layer_units = x_train.shape[1])
            # Use GPU if available
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            network = PyTorchNetwork(layers_for_network).to(device)
            network.eval()

            print("~"*50)
            print(network.parameters())
            print("~"*50)

            other_params_dict = PYTORCHUTILS.get_other_pytorch_params(nnptc_params, task_type = "CLASSIFICATION", network_params = network.parameters())

            print("~"*50)
            print("NNPTC-PARAMS - ", nnptc_params)
            print("~"*50)
            print("OTHER-PARAMS-DICT - ", other_params_dict)
            print("~"*50)
            print("NEURAL-NETWORK - ", network)
            print("~"*50)

            criterion = other_params_dict["loss_criterion"]
            n_epochs = other_params_dict["number_of_epochs"]
            batch_size = other_params_dict["batch_size"]
            optimizer = other_params_dict["optimizer"]
            optimizer = optim.Adam(network.parameters(), weight_decay=0.0001)

            dataloader_params = {
            "batch_size": batch_size,
            "shuffle": True
            # "num_workers":
            }

            train_loader = torch_data_utils.DataLoader(trainset, **dataloader_params)
            test_loader = torch_data_utils.DataLoader(testset, **dataloader_params)

            '''
            Training the network;
            Batchnormalization(num_features) should be equal to units_op for that layer in training config;
            else --> RuntimeError('running_mean should contain 100 elements not 200',)
            '''

            for epoch in range(n_epochs):
                batchwise_losses = []
                average_loss = 0.0

                for i, (inputs, labels) in enumerate(train_loader):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # Zero the parameter gradients
                    optimizer.zero_grad()

                    # Forward + backward + optimize
                    outputs = network(inputs.float())
                    loss = criterion(outputs, labels.long())
                    loss.backward()
                    optimizer.step()

                    average_loss += loss.item()
                    batchwise_losses.append(loss.item())

                average_loss_per_epoch = old_div(average_loss,(i + 1))
                print("+"*80)
                print("EPOCH - ", epoch)
                print("BATCHWISE_LOSSES shape - ", len(batchwise_losses))
                print("AVERAGE LOSS PER EPOCH - ", average_loss_per_epoch)
                print("+"*80)

            trainingTime = time.time()-st
            bestEstimator = network

            outputs_x_test_tensored = network(x_test_tensored.float())
            softmax = torch.nn.Softmax()
            y_prob = softmax(outputs_x_test_tensored)

            _, y_score = torch.max(outputs_x_test_tensored, 1)

            y_score = y_score.tolist()
            y_prob = y_prob.tolist()